# Abstract

LettuceDetect is a new lightweight, open-source framework designed to identify hallucinations in Retrieval-Augmented Generation (RAG) systems.

This framework addresses two critical limitations of existing methods by utilizing ModernBERT's extended context capabilities and being significantly more computationally efficient than comparable Large Language Model (LLM)-based approaches:
- Context window constraints of traditional encoder-based methods

- Computational inefficiency of LLM-based approaches

Evaluated on the RAGTruth benchmark dataset, LettuceDetect outperforms previous encoder-based models and most prompt-based models in detecting unsupported claims at a token level.

# 2. HallucinationDetector class

I've documented the class HallucinationDetector.

## 2.1 The constructor

`def __init__(self, method: str = "transformer", **kwargs)`

The constructor of the HallucinationDetector class accepts an argument called method of type string. This argument can take the following values:

- `"transformer"`: In this case, the constructor delegates the task to the class TransformerDetector

- `"llm"`: In this case, the constructor delegates the task to the class "LLMDetector"

### 2.1.1 The TransformerDetector class
```
def __init__(
        self,
        model_path: str,
        max_length: int = 4096,
        device=None,
        lang: Lang = "en",
        **tok_kwargs
):
```

- `model_path`: Path to the pre-trained model. The full list of pretrained BERT transformers is available [here](https://huggingface.co/KRLabsOrg).
- `max_length`: Maximum length of the input sequence.
- `device`: Device to use for inference. If `None`, by default will be `torch.device("cuda")` or `torch.device("cpu")`
- `lang`: Language of the model. Supported languages:
    ```
    "en": "English",
    "de": "German",
    "fr": "French",
    "es": "Spanish",
    "it": "Italian",
    "pl": "Polish",
    "cn": "Chinese",
    ```
- `tok_kwargs`: Additional keyword arguments for the tokenizer.



### 2.1.2 The LLMDetector class
```
def __init__(
        self,
        model: str = "gpt-4.1-mini",
        temperature: float = 0.0,
        lang: Lang = "en",
        zero_shot: bool = False,
        fewshot_path: str | None = None,
        prompt_path: str | None = None,
        cache_file: str | None = None,
):
```

- `model`: The model to use for hallucination detection.
- `temperature`: The temperature to use for hallucination detection.
- `lang`: The language to use for hallucination detection (cf. the previous language list).
- `zero_shot`: Whether to use zero-shot hallucination detection.
- `fewshot_path`: The path to the few-shot examples.
- `prompt_path`: The path to the prompt.
- `cache_file`: The path to the cache file.

When using the LLMDetector, only the 'spans' output format is supported.

## 2.2 The predict function

```
def predict(
    self,
    context: list[str],
    answer: str,
    question: str | None = None,
    output_format: str = "tokens",
) -> list:
```

This function predicts hallucination tokens or spans given passages and an answer.

- context: List of passages that were supplied to the LLM / user.
- answer: Model‑generated answer to inspect.
- question: Original question (``None`` for summarisation).
- output_format: ``"tokens"`` for token‑level dicts, ``"spans"`` for character spans.

It first uses `PromptUtils.format_context()` to format the context and question intro a single prompt and then it uses the internal `_predict()` function to convert a single (prompt, answer) pair into hallucination spans.

### 2.2.1 Comments on the predict function of the LLMDetector class

Two important things to note for this case:

1. The specified output format can only be `"spans"`. Otherwise, an exception will be thrown.

2. This function is written so it uses the OpenAI API. However, modifying the code in order to use another LLM doesn't seem very difficult.

### 2.2.2 The output format
For the span format:
```
[{
    'text': str,        # The hallucinated text
    'start': int,       # Start position in answer
    'end': int,         # End position in answer
    'confidence': float # Model's confidence (0-1)
}]
```

For the token format (only available for the TransformerDetector class):
```
[{
    'token': str,       # The token
    'pred': int,        # 0: supported, 1: hallucinated
    'prob': float       # Model's confidence (0-1)
}]
```

# 3. Demos

The following demo is extracted from the original GitHub repository of the project:

In [1]:
%pip install lettucedetect

Note: you may need to restart the kernel to use updated packages.


In [4]:
contexts = [
    "France is a country in Europe. The capital of France is Paris. The population of France is 67 million.",
]
question = "What is the capital of France? What is the population of France?"
answer = "The capital of France is Paris. The population of France is 67 million."

In [5]:
from lettucedetect.models.inference import HallucinationDetector

# Transformer-based detector
detector = HallucinationDetector(
    method="transformer", model_path="KRLabsOrg/lettucedect-base-modernbert-en-v1"
)

# Then predict the same way
predictions = detector.predict(context=contexts, question=question, answer=answer, output_format="spans")
predictions

[]

# 4. What's next

Possible ideas for using this framework:

- Try to modify the prediction function so it uses a different LLM API (like Azure OpenAI).

- Research how to use the detector in combination with a RAG system that uses a vector database (like ChromaDB or FAISS)

# 5. References

Paper - [LettuceDetect: A Hallucination Detection Framework for RAG Applications](https://arxiv.org/abs/2502.17125)

GitHub repository - [LettuceDetect](https://github.com/KRLabsOrg/LettuceDetect)